In [ ]:
# =========================================
# 🧠 Setup môi trường Ollama + Streamlit trong Google Colab (Windows user compatible)
# =========================================

!apt-get update -y
!apt-get install -y libcurl4-openssl-dev

# Cài Ollama
!curl -fsSL https://ollama.com/install.sh | sh

# Cài các thư viện Python cần thiết
!pip install streamlit pyngrok ollama



'apt-get' is not recognized as an internal or external command,
operable program or batch file.
'apt-get' is not recognized as an internal or external command,
operable program or batch file.
'sh' is not recognized as an internal or external command,
operable program or batch file.


  Using cached ollama-0.6.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached altair-5.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached click-8.3.0-py3-none-any.whl.metadata (2.6 kB)
     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     ------------------- ------------------ 30.7/60.9 kB 660.6 kB/s eta 0:00:01
     ------------------------------- ------ 51.2/60.9 kB 525.1 kB/s eta 0:00:01
     -------------------------------------- 60.9/60.9 kB 460.9 kB/s eta 0:00:00
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
     ---------------------------------------- 0.0/44.3 kB ? eta -:--:--
     ---------------------------------------- 44.3/44.3 kB 1.1 MB/s eta 0:00:00
  Using cached gitpython-3.1.45-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
     ------------------


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Kill bất kỳ Ollama nào đang chạy
!pkill ollama || echo "Ollama not running"

# Chạy Ollama serve nền và redirect log
!nohup ollama serve > /tmp/ollama.log 2>&1 &

# Đợi server khởi động
import time
time.sleep(5)

# Kiểm tra API
!curl http://127.0.0.1:11434/api/tags

!ollama pull llama3:8b

'pkill' is not recognized as an internal or external command,
operable program or batch file.


"Ollama not running"


OSError: Background processes not supported.

In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token("352P7oGjrqdMRQytZvuLuZWUPqV_5NgX4Gy3uYtH5ETAMq7fL")

PermissionError: [WinError 5] Access is denied

In [ ]:
%%writefile app.py
import streamlit as st
import json
import ollama
from datetime import datetime

# ================== CẤU HÌNH OLLAMA ==================
ollama.api_base = "http://127.0.0.1:11434"
MODEL_NAME = "llama3:8b"  # Model đã pull

# ================== SESSION STATE ==================
if "logged_in" not in st.session_state:
    st.session_state.logged_in = False
if "username" not in st.session_state:
    st.session_state.username = ""
if "messages" not in st.session_state:
    st.session_state.messages = []  # Lịch sử chat tạm thời

# ================== LOGIN FORM ==================
if not st.session_state.logged_in:
    st.title("🔐 Login")
    username = st.text_input("Username")
    password = st.text_input("Password", type="password")

    if st.button("Login"):
        if username == "user" and password == "123":
            st.session_state.logged_in = True
            st.session_state.username = username
            st.success("✅ Logged in successfully!")
        else:
            st.error("❌ Invalid credentials")

# ================== MAIN APP ==================
else:
    st.title("🌍 Travel Itinerary Generator")

    tab1, tab2 = st.tabs(["✈️ Generate Itinerary", "💬 Chat with Ollama"])

    # ================== TAB 1: ITINERARY GENERATOR ==================
    with tab1:
        origin = st.text_input("Origin city")
        destination = st.text_input("Destination city")
        dates = st.text_input("Travel dates (e.g., 2025-11-10 to 2025-11-15)")
        interests = st.multiselect("Interests", ["food", "museums", "nature", "nightlife", "shopping"])
        pace = st.radio("Travel pace", ["relaxed", "normal", "tight"])

        if st.button("🚀 Generate Itinerary"):
            with st.spinner("🧠 Ollama is generating your itinerary..."):
                prompt = f"""
                Create a day-by-day travel itinerary from {origin} to {destination}
                on dates {dates}, interested in {', '.join(interests)} with {pace} pace.
                Return only a valid JSON object with the structure:
                {{
                  "Day 1": {{"morning":"...","afternoon":"...","evening":"..."}},
                  "Day 2": ...
                }}
                """
                try:
                    response = ollama.chat(model=MODEL_NAME, messages=[{"role": "user", "content": prompt}])
                    response_text = response["message"]["content"]

                    # Thống kê tốc độ
                    count = response.get("eval_count", 0)
                    duration = response.get("eval_duration", 1) / 1e9
                    speed = round(count / duration, 1)
                    st.caption(f"⚡ {count} tokens in {duration:.2f}s ({speed} tokens/s)")

                except Exception as e:
                    st.error(f"❌ Ollama error: {e}")
                    response_text = "{}"

            # Parse JSON
            itinerary = {}
            start = response_text.find("{")
            end = response_text.rfind("}")
            json_part = response_text[start:end+1] if start != -1 and end != -1 else "{}"

            try:
                itinerary = json.loads(json_part)
            except json.JSONDecodeError:
                st.warning("⚠️ Model output wasn't valid JSON. Showing raw response instead.")
                st.text_area("Raw output from model", response_text, height=200)

            # Hiển thị kết quả
            if itinerary:
                st.subheader("🗓️ Day-by-Day Itinerary")
                for day, plan in itinerary.items():
                    st.markdown(f"### {day}")
                    for time, activity in plan.items():
                        st.markdown(f"- **{time.capitalize()}**: {activity}")

            # ================== LƯU LỊCH SỬ ITINERARY ==================
            history_file = "history.json"
            try:
                with open(history_file, "r", encoding="utf-8") as f:
                    history = json.load(f)
            except FileNotFoundError:
                history = []

            history.append({
                "user": st.session_state.username,
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "type": "itinerary",
                "input": {
                    "origin": origin,
                    "destination": destination,
                    "dates": dates,
                    "interests": interests,
                    "pace": pace
                },
                "raw_response": response_text,
                "itinerary": itinerary
            })

            with open(history_file, "w", encoding="utf-8") as f:
                json.dump(history, f, indent=2, ensure_ascii=False)

            st.success("✅ Itinerary saved to history!")

        # Xem lịch sử itinerary
        if st.checkbox("📜 Show itinerary history"):
            history_file = "history.json"
            try:
                with open(history_file, "r", encoding="utf-8") as f:
                    history = json.load(f)
            except FileNotFoundError:
                history = []

            user_history = [h for h in history if h.get("user") == st.session_state.username and h.get("type")=="itinerary"]
            if not user_history:
                st.info("No itinerary history yet for this user.")
            else:
                for chat in reversed(user_history):
                    timestamp = chat.get("timestamp", "Unknown time")
                    origin = chat.get("input", {}).get("origin", "Unknown")
                    destination = chat.get("input", {}).get("destination", "Unknown")
                    with st.expander(f"🕓 {timestamp} — {origin} → {destination}"):
                        st.json(chat.get("input", {}))
                        if chat.get("itinerary"):
                            st.json(chat["itinerary"])
                        else:
                            st.text_area(
                              "Raw response",
                              chat.get("raw_response", ""),
                              height=150,
                              key=f"raw_response_{timestamp}_{hash(str(chat))}"
                            )
    # ================== TAB 2: CHAT ==================
    with tab2:
        st.subheader("💬 Chat with Ollama")

        # Hiển thị lịch sử chat hiện tại
        for msg in st.session_state.messages:
            st.chat_message(msg["role"]).markdown(msg["content"])

        user_input = st.chat_input("Say something to Ollama...")

        if user_input:
            st.session_state.messages.append({"role": "user", "content": user_input})
            st.chat_message("user").markdown(user_input)

            with st.spinner("🤖 Ollama is thinking..."):
                ollama_messages = [{"role": m["role"], "content": m["content"]} for m in st.session_state.messages]
                try:
                    response = ollama.chat(model=MODEL_NAME, messages=ollama_messages)
                    reply = response["message"]["content"]

                    count = response.get("eval_count", 0)
                    duration = response.get("eval_duration", 1) / 1e9
                    speed = round(count / duration, 1)
                    stats = f"⚡ {count} tokens in {duration:.2f}s ({speed} tokens/s)"
                except Exception as e:
                    reply = f"❌ Ollama error: {e}"
                    stats = ""

            st.session_state.messages.append({"role": "assistant", "content": reply})
            st.chat_message("assistant").markdown(reply)
            if stats:
                st.caption(stats)

            # ================== LƯU LỊCH SỬ CHAT ==================
            history_file = "history.json"
            try:
                with open(history_file, "r", encoding="utf-8") as f:
                    history = json.load(f)
            except FileNotFoundError:
                history = []

            history.append({
                "user": st.session_state.username,
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "type": "chat",
                "message": user_input,
                "response": reply
            })

            with open(history_file, "w", encoding="utf-8") as f:
                json.dump(history, f, indent=2, ensure_ascii=False)

        # Xem lịch sử chat
        if st.checkbox("📜 Show chat history"):
            history_file = "history.json"
            try:
                with open(history_file, "r", encoding="utf-8") as f:
                    history = json.load(f)
            except FileNotFoundError:
                history = []

            user_chat_history = [h for h in history if h.get("user")==st.session_state.username and h.get("type")=="chat"]
            if not user_chat_history:
                st.info("No chat history yet for this user.")
            else:
                for chat in reversed(user_chat_history):
                    timestamp = chat.get("timestamp", "Unknown time")
                    message = chat.get("message", "")
                    response_text = chat.get("response", "")
                    with st.expander(f"🕓 {timestamp}"):
                        st.markdown(f"**You:** {message}")
                        st.markdown(f"**Ollama:** {response_text}")


In [ ]:
from pyngrok import ngrok

# Ensure no ngrok processes are running
ngrok.kill()

# Tạo đường dẫn public
public_url = ngrok.connect(8501)
print("🌐 App running at:", public_url)

# Chạy streamlit
!streamlit run app.py --server.port 8501 &>/dev/null&